In [1]:
from gensim.models import Word2Vec
from gensim import models
import gensim
import pandas as pd
import os
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import numpy as np  # Make sure that numpy is imported
from sklearn.cluster import KMeans
import time

model = Word2Vec.load("300features_40minwords_10context")

D:\Anaconda\envs\PythonData\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [2]:
model.most_similar("awful")

D:\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7653412818908691),
 ('horrible', 0.7354335188865662),
 ('abysmal', 0.7140626907348633),
 ('dreadful', 0.713903546333313),
 ('atrocious', 0.7132828235626221),
 ('horrendous', 0.6752372980117798),
 ('horrid', 0.6706846356391907),
 ('appalling', 0.6625896692276001),
 ('laughable', 0.6240285634994507),
 ('amateurish', 0.6178346872329712)]

In [3]:
model.wv.syn0.shape

D:\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


(16490, 300)

In [4]:
model["flower"]

D:\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-9.40895155e-02,  7.75069883e-03, -4.43649292e-02, -2.99042650e-02,
       -5.17895482e-02,  4.81346585e-02, -5.68473637e-02, -9.62749943e-02,
       -1.24056647e-02, -1.51469475e-02,  8.59379917e-02, -5.55642098e-02,
       -5.23529500e-02, -1.02777071e-02, -1.22859791e-01, -2.55358815e-02,
       -4.69389185e-02,  4.37662229e-02, -6.43464923e-02,  5.12221307e-02,
        6.98277429e-02, -5.01920953e-02,  8.77969936e-02,  4.34962921e-02,
        6.99577630e-02,  3.65460292e-02,  6.50715665e-04, -3.90566140e-02,
        4.43493389e-02, -1.33912131e-01,  1.05285764e-01,  6.07794151e-02,
        1.01269968e-01,  9.50731430e-03,  6.89700916e-02,  1.07852658e-02,
        1.77229487e-03,  1.78899504e-02, -4.50990945e-02,  7.01271091e-03,
        1.96310431e-02,  4.35354523e-02, -1.55095076e-02,  3.85648035e-03,
        3.65503132e-02, -7.81727508e-02, -1.87888350e-02, -1.26106113e-01,
       -1.69923995e-02,  1.16207078e-01, -2.41300184e-02,  1.34462658e-02,
       -6.77878559e-02,  

In [5]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given paragraph
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in the model's vocabulary
    index2word_set = set(model.wv.index2word)
    #
    # Loop through words, if it is in the model's vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [6]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    counter = 0
    counter = int(counter)
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    # Loop through the reviews
    for review in reviews:
       # Print a status message every 1000th review
        if counter%1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
        
       # Call the function (defined above) that makes average feature vectors, increment the counter
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
        
    return reviewFeatureVecs

In [7]:
train = pd.read_csv('labeledTrainData.tsv\labeledTrainData.tsv', header=0, delimiter="\t", quoting=3 )
test = pd.read_csv('testData.tsv', header=0, delimiter="\t", quoting=3 )

In [8]:
num_features = 300 # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4 # Threads
context = 10 # Context window size
downsampling = 1e-3 # Downsampling setting fro frequent words

In [9]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    review_text = BeautifulSoup(review,"lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    
    # Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return(words)

In [10]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Use stop word removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000


D:\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [11]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )
result = forest.predict( testDataVecs )

# Test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [12]:
print(forest.predict( testDataVecs ))
print(classification_report(output['sentiment'], result))
print(confusion_matrix(output['sentiment'], result))

[1 0 1 ... 1 1 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12341
           1       1.00      1.00      1.00     12659

   micro avg       1.00      1.00      1.00     25000
   macro avg       1.00      1.00      1.00     25000
weighted avg       1.00      1.00      1.00     25000

[[12341     0]
 [    0 12659]]


In [13]:
start = time.time() 
# Set "k" (num_clusters) to be average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = int(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

D:\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


Time taken for K Means clustering:  623.7197349071503 seconds.


In [14]:
# Create a Word / Index dictionary, mapping each vocabulary word to a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [15]:
# For the first 10 clusters
for cluster in range(0,15):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(list(word_centroid_map.values()))):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map)[i])
    print(words)


Cluster 0
['solidly']

Cluster 1
['carla', 'pauline', 'rosa', 'manu', 'elena', 'auteuil']

Cluster 2
['beethoven', 'barber']

Cluster 3
['narrated', 'guided']

Cluster 4
['un', 'similarly', 'stunningly', 'astonishingly', 'strikingly', 'portraits', 'overwhelmingly']

Cluster 5
['jury', 'election', 'coup', 'presidential', 'council', 'division', 'candidates', 'reserve', 'sisko', 'amendment']

Cluster 6
['sights', 'pipes']

Cluster 7
['prejudices', 'assumptions', 'sympathies']

Cluster 8
['maverick', 'stagecoach', 'elevated', 'bray', 'tumble']

Cluster 9
['vanessa', 'dakota', 'fanning', 'paige', 'chloe', 'lucinda']

Cluster 10
['sabotage', 'resurrect', 'thwart']

Cluster 11
['dialogs', 'dialogues', 'scripting']

Cluster 12
['strained', 'articulate', 'profane', 'overplayed', 'talkative', 'unintelligent', 'notoriously', 'didactic', 'unsophisticated', 'moralizing', 'disingenuous']

Cluster 13
['premier', 'january', 'flopped']

Cluster 14
['bitch', 'stripper', 'hunk', 'bimbo', 'slut', 'weirdo

In [16]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1

    return bag_of_centroids

In [17]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [18]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [19]:
print(result)
print(classification_report(output['sentiment'], result))
print(confusion_matrix(output['sentiment'], result))

[1 0 1 ... 0 1 1]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12871
           1       1.00      1.00      1.00     12129

   micro avg       1.00      1.00      1.00     25000
   macro avg       1.00      1.00      1.00     25000
weighted avg       1.00      1.00      1.00     25000

[[12871     0]
 [    0 12129]]
